In [3]:
# Importing packages
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import numpy as np
from ipywidgets import Layout
import seaborn as sns
import math

import time
from scipy import linalg
from scipy import optimize
import sympy as sm

from tkinter import *
import tkinter as tk

In [4]:
## Importing and cleaning the data 

filename = 'imdb.csv'

## Defining the function to import and clean the data
def gen_df(filename):
    """ Returns a pandas dataframe containing information about movies from the IMDB database 

    Args:
    filename (csv-file): The csv-file have to be the file imdb.csv

    Returns:
    A pandas dataframe

    Notice:
    The function will not work if other files than imdb.csv are used as input """
        
    # Get .csv.file
    data = pd.read_csv(filename, sep=';', encoding='latin-1', escapechar='\\')

    # Read file into pandas dataframe
    df = pd.DataFrame(data)

    # Drop unwanted columns
    for i in range(44,48):
        df.drop(columns=[f'Unnamed: {i}'], inplace=True)
    df.drop(columns=['fn','wordsInTitle','url'], inplace=True)

    # Keep only observations of 'movie'-type
    I = df['type'] == 'video.movie'
    df = df.loc[I]
    df.drop(columns=['type'], inplace=True)

    # Drop movies with missing observations
    df.dropna(inplace=True)

    # During the importing process, the first decimal has become 0 for all movies.
    # Thus, we replace "0"'s with ""
    df['imdbRating'] = df['imdbRating'].astype(str)
    df['imdbRating'].replace(regex=True, inplace=True,to_replace='0',value='')
    df['imdbRating'] = df['imdbRating'].astype(float)

    # Transform duration from seconds to hours
    df['duration'] = df['duration']/60**2

    # Drop years before 1920 and the year 2014 because of few obervations
    I = (df['year']>=1920) & (df['year']<=2013)
    df = df.loc[I]

    # Change the type of 'year' to integer
    df['year'] = df['year'].astype(int)    
    
    # Sort observations and reset index
    df.sort_values('year', inplace=True)
    df.reset_index(inplace=True)

    # Generating a variable that shows the decade (as a string)
    year_list = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010]

    df['decade'] = ''
    for i,start in enumerate(year_list):
        end = start+10
        df.loc[(df['year'] >= start) & (df['year'] < end), 'decade'] = f'{year_list[i]}s'

    return df

df = gen_df(filename)

In [35]:
## Pretty Printing our functions 

# Defining symbols 
sm.init_printing(use_unicode=True) # for pretty printing

# Parameters
alphak = sm.symbols("alpha_k")
betak = sm.symbols("beta_k")
delta = sm.symbols("delta")
gamma = sm.symbols("gamma")
rho1 = sm.symbols("rho_1")
rho2 = sm.symbols("rho_2")
omega = sm.symbols("omega")

#Variables
Ui = sm.symbols("U_i")
Gki = sm.symbols("G_ki")
Dki = sm.symbols("D_ki")
Ni = sm.symbols("N_i")
Wi = sm.symbols("W_i")
Li = sm.symbols("L_i")
Li_sqr = sm.symbols("L_i^2")
Ri_model = sm.symbols("R_i^model")
Ri_data = sm.symbols("R_i^data")
x = sm.symbols("x")

k = sm.symbols("k")
n = sm.symbols("n")


# Functions
sm.summation(alphak*Gki, (k, 1, n))



# sm.init_printing(use_unicode=False)

'$\\alpha_i > \\beta_i$'